In [1]:
import xgboost
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import PSID
from PSID.evaluation import evalPrediction

In [2]:
PATH_FEATURES = "/home/lauraflyra/Documents/BCCN/Lab_Rotation_USC/Code/Data/py_neuromodulation_derivatives/sub-000_ses-right_task-force_run-3/sub-000_ses-right_task-force_run-3_FEATURES.csv"
data_features = pd.read_csv(PATH_FEATURES, index_col=0)

# ECOG

In [3]:
ecog_feature_df = data_features.filter(like='ECOG').filter(like='bandpass_activity').to_numpy()
behavior_df = data_features["MOV_LEFT_CLEAN"].to_numpy().reshape(-1, 1)

In [4]:
train_idx_features = np.arange(np.round(0.5 * data_features.shape[0]), dtype=int)
test_idx_features = np.arange(1 + train_idx_features[-1], data_features.shape[0])

ecog_feat_train = ecog_feature_df[train_idx_features]
ecog_feat_test = ecog_feature_df[test_idx_features]

mov_features_train = behavior_df[train_idx_features]
mov_features_test = behavior_df[test_idx_features]

In [5]:
model = xgboost.XGBRegressor()
model.fit(ecog_feat_train, mov_features_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [6]:
mov_feat_test_pred = model.predict(ecog_feat_test)

In [7]:
r2 = r2_score(mov_features_test, mov_feat_test_pred)
print("R2: {}".format(r2))

R2: 0.6600606511063383


In [8]:
R = np.corrcoef(mov_features_test, mov_feat_test_pred, rowvar=False)

In [9]:
R

array([[1.       , 0.8156097],
       [0.8156097, 1.       ]])

In [10]:
idSys_ecog = PSID.PSID(ecog_feat_train, mov_features_train, 8, 8, 8)
mov_features_test_pred_ecog, feat_test_pred_ecog, x_feat_test_pred_ecog = idSys_ecog.predict(ecog_feat_test)
R2_feat = evalPrediction(mov_features_test, mov_features_test_pred_ecog, "R2")
print('Behavior decoding R2:\n  PSID => {:.3g}'.format(np.mean(R2_feat)))

Behavior decoding R2:
  PSID => 0.667


In [11]:
cv_score_xgboost_ecog = cross_val_score(model, X = ecog_feature_df, y=behavior_df, scoring='r2', cv = 5, n_jobs=-1 )

In [12]:
print(np.mean(cv_score_xgboost_ecog), np.std(cv_score_xgboost_ecog))

0.6517765346236696 0.03832398477198442


In [13]:
from src.cross_validation_PSID import cross_validation

In [14]:
cv_score_psid_ecog = cross_validation(ecog_feature_df, behavior_df,5, 8, 8, 8)

Behavior decoding evaluation:
  PSID => 0.613
Behavior decoding evaluation:
  PSID => 0.715
Behavior decoding evaluation:
  PSID => 0.715
Behavior decoding evaluation:
  PSID => 0.641
Behavior decoding evaluation:
  PSID => 0.7


In [15]:
print(np.mean(cv_score_psid_ecog), np.std(cv_score_psid_ecog))

0.6766908342902417 0.04204777803628487


# LFP

In [16]:
lfp_feature_df = data_features.filter(like='LFP').filter(like='bandpass_activity').to_numpy()
behavior_df = data_features["MOV_LEFT_CLEAN"].to_numpy().reshape(-1, 1)
train_idx_features = np.arange(np.round(0.5 * data_features.shape[0]), dtype=int)
test_idx_features = np.arange(1 + train_idx_features[-1], data_features.shape[0])

lfp_feat_train = lfp_feature_df[train_idx_features]
lfp_feat_test = lfp_feature_df[test_idx_features]

mov_features_train = behavior_df[train_idx_features]
mov_features_test = behavior_df[test_idx_features]
model_lfp = xgboost.XGBRegressor()
model_lfp.fit(lfp_feat_train, mov_features_train)
mov_feat_test_pred_lfp = model_lfp.predict(lfp_feat_test)
r2_lfp = r2_score(mov_features_test, mov_feat_test_pred_lfp)
print("R2: {}".format(r2_lfp))
R = np.corrcoef(mov_features_test, mov_feat_test_pred_lfp, rowvar=False)

R2: 0.1264917963498534


In [17]:
R

array([[1.        , 0.41407521],
       [0.41407521, 1.        ]])

In [18]:
idSys_lfp = PSID.PSID(lfp_feat_train, mov_features_train, 8, 8, 8)
mov_features_test_pred_lfp, feat_test_pred_lfp, x_feat_test_pred_lfp = idSys_lfp.predict(lfp_feat_test)
R2_feat_lfp = evalPrediction(mov_features_test, mov_features_test_pred_lfp, "R2")
print('Behavior decoding R2:\n  PSID => {:.3g}'.format(np.mean(R2_feat_lfp)))

Behavior decoding R2:
  PSID => 0.257


In [19]:
cv_score_xgboost_lfp = cross_val_score(model, X = lfp_feature_df, y=behavior_df, scoring='r2', cv = 5, n_jobs=-1 )

In [20]:
print(np.mean(cv_score_xgboost_lfp), np.std(cv_score_xgboost_lfp))

0.06078531953545767 0.02199741224863082


In [21]:
cv_score_psid_lfp = cross_validation(lfp_feature_df, behavior_df,5, 8, 8, 8)

Behavior decoding evaluation:
  PSID => 0.435
Behavior decoding evaluation:
  PSID => 0.455
Behavior decoding evaluation:
  PSID => 0.275
Behavior decoding evaluation:
  PSID => 0.312
Behavior decoding evaluation:
  PSID => 0.25


In [22]:
print(np.mean(cv_score_psid_lfp), np.std(cv_score_psid_lfp))

0.3453615770352744 0.08405291829358978
